In [1]:
"""Mainly Edited for private usage by:  Ioannis Agriomallos
                                        Ioanna Mitsioni
License: BSD 3 clause

============= CURRENT CODE USAGE =============
Current code trains MLP Classifiers, to classify force input samples as stable (0) or slip (1)
---- Input
-> Input samples originate from optoforce sensors and are 3D (fx,fy,fz) and come from 2 different datasets, 
   one training, containing several surfaces as well as slip-stable occurrences, 
   and one validation, containing 1 surface with slip-stable occurrences on a completely unseen task-setup.
---- Input transformation
-> Several pre-features can be taken from these inputs, but here |f| is kept.
-> Several time and frequency domain features are extracted from pre-feature windows. 
  (implemented in 'featext.py') These windows have size w and are shifted by s on each sample
-> Then a feature selection-ranking is performed using MutualVariableInformation
-> Finally PCA is performed to keep a reduced set among the best selected features
---- Training of ML Classifiers
-> Several MLP Classifiers are trained for all combinations of selected featuresets-datasets
---- Results
-> Stats of classification results are kept inside each .npz along with the respective trained model
"""
print(__doc__)
import time
start_time = time.time()
import numpy as np
from copy import deepcopy
from ml_training import *
import matplotlib.pyplot as plt
%matplotlib inline 
# %matplotlib qt
# inline (suitable for ipython only, shown inside browser!) or qt (suitable in general, shown in external window!)
from matplotlib.colors import ListedColormap
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D

Mainly Edited for private usage by:  Ioannis Agriomallos
                                        Ioanna Mitsioni
License: BSD 3 clause

============= CURRENT CODE USAGE =============
Current code trains MLP Classifiers, to classify force input samples as stable (0) or slip (1)
---- Input
-> Input samples originate from optoforce sensors and are 3D (fx,fy,fz) and come from 2 different datasets, 
   one training, containing several surfaces as well as slip-stable occurrences, 
   and one validation, containing 1 surface with slip-stable occurrences on a completely unseen task-setup.
---- Input transformation
-> Several pre-features can be taken from these inputs, but here |f| is kept.
-> Several time and frequency domain features are extracted from pre-feature windows. 
  (implemented in 'featext.py') These windows have size w and are shifted by s on each sample
-> Then a feature selection-ranking is performed using MutualVariableInformation
-> Finally PCA is performed to keep a reduced 

In [2]:
class struct:
    def __init__(self):
        ####### TRAINING DEFAULTS
        self.cv = KFold(n_splits=5,random_state=42)
        self.scaler = StandardScaler() ;
        self.decomp = PCA(n_components=20)
        self.names = ["NearNb", "RBFSVM1", "MLP1", "RandFor"]
        self.classifiers = [KNeighborsClassifier(5),
                       SVC(gamma='auto', C=1),
                       MLPClassifier(solver='lbfgs',alpha=1e-4,hidden_layer_sizes=(10,10),random_state=1,verbose=True),
                       RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)]

        self.download = 0            # Download pre-computed (1) data or compute them all anew (0)
        self.delete_big_features = 1 # Delete (1) or keep (0) computed big-in-size features,
                                  # helping mainly to avoid several computations when recomputing features

        ############ INITIALISATION PARAMETERS ############
        self.window, self.shift = 1024, 20
        self.samplesperdataset = 10000
        self.havelabel = 1
        self.returntime = 0
        self.featlabel = 0         # 0: all features, 1: temporal, 2: frequency, 3: FFT only
        self.magnFFT = 0           # 0: FFT in magnitude format, 1: FFT in real and imag format,
        self.featall = 0           # 0: all, 1: feat1 (phinyomark's), 2: feat2 (golz's)
        self.CV = 5                # cross validation checks
        self.numfeat = 10          # number of features to show
        self.nfeat = 1000          # number of features to keep
        ###### Initialize necessary names and paths
        self.datapath = 'data/'
        self.datafile = self.datapath+'dataset.npz'
        self.validfile = self.datapath+'validation.mat'
        self.featpath = self.datapath+'features/'+str(self.window)+'_'+str(self.shift)+'/'
        self.allfeatpath = self.featpath+'AllFeatures/'
        self.prefeatname = 'prefeatures_scaled'+'_'+str(self.window)+'_'+str(self.shift)+'_'+str(self.samplesperdataset)
        self.prefeatfile = self.featpath+self.prefeatname+'.npz'
        self.featname = 'features_scaled'+'_'+str(self.window)+'_'+str(self.shift)+'_'+str(self.samplesperdataset)
        self.featfile = self.featpath+self.featname+'.npz'
        self.validfeatname = 'valid'+self.featname
        self.validfeatfile = self.featpath+self.validfeatname+'.npz'
        self.surffile = self.featpath+self.featname+'_2fing_6surf.npz'
        self.XYfile = self.featpath+self.featname+'_XY.npz'
        self.XYsplitfile = self.featpath+self.featname+'_XYsplit.npz'
        self.validsurffile = self.featpath+self.validfeatname+'_2fing_6surf.npz'
        self.validXYfile = self.featpath+self.validfeatname+'_XY.npz'
        self.validXYsplitfile = self.featpath+self.validfeatname+'_XYsplit.npz'
        self.respath = self.datapath+'scaled_results'
        self.toolfile = self.datapath+'bargraph.zip'
        self.toolpath = self.datapath+'bargraph-rel_4_8/'
        self.tool = './'+self.toolpath+'bargraph.pl'
######### INITIALIZE OBJECT-STRUCT WITH PARAMETERS AND PASS THEM TO ML MODULE ########
c = struct()
m = ml(c)

In [ ]:
######### DOWNLOAD NECESSARY FILES ###########
download_required_files()

Necessary  data/dataset.npz  already here!
Necessary  data/validation.mat  already here!
Downloaded 575.7 MB of content in total!


In [ ]:
############ TRAINING PROCEDURE ##############
# necessary steps before training
f,l,fd,member,m1,m2 = data_prep(c.datafile)                      # read input force and labels
### BEGIN SCALING 
tf = deepcopy(f)
scales = [0.1, 0.5, 1.0, 2.0, 5.0, 20.0]
for sc in scales:
    for i in range(len(f)):
        tf[i][:,:-1] = sc * deepcopy(f[i][:,:-1])
#     print tf.shape
    tf = np.concatenate((deepcopy(f),tf),axis=0)
#     print tf.shape, f.shape, np.max(tf[0][:,:-1]), np.max(f[0][:,:-1])
tf = tf[len(f):]
tm = np.ones(len(f)*len(scales))*member[0]/(len(scales)*1.)
m1, m2 = len(scales)*m1, len(scales)*m2
print '6 -> scaled f: ', tf.shape, tm.shape
# for i in range(len(tf)/len(f)):
#     print i,np.max(tf[i*len(f)][:,:-1])
### FINISH SCALING
prefeat = compute_prefeat(tf)                                    # compute corresponding prefeatures
features, labels = feature_extraction(prefeat, tm, c.featfile,
                                      c.featname+'_')            # feature extraction from prefeatures
# avg_feat_comp_time(prefeat)                                      # average feature extraction time
new_labels = label_cleaning(prefeat,labels,tm)                   # trim labels, around change points
X,Y,Yn,Xsp,Ysp = computeXY(features,labels,new_labels,m1,m2,
                           c.XYfile,c.XYsplitfile)               # compute data and labels, trimmed and untrimmed
surf, surfla = computeXY_persurf(Xsp,Ysp,c.surffile)             # compute per surface data and labels
# training and offline testing
train_1_surface(surf,surfla)                                     # training of all combinations per 1 surface
train_2_surface(surf,surfla)                                     # training of all combinations per 2 surfaces
train_3_surface(surf,surfla)                                     # training of all combinations per 3 surfaces
train_4_surface(surf,surfla)                                     # training of all combinations per 4 surfaces
train_5_surface(surf,surfla)                                     # training of all combinations per 5 surfaces

---------------------------- LOADING DATA and COMPUTING NECESSARY STRUCTS ----------------------------
1 -> f1: (36,) (36,) (36, 4)
2 -> f2: (36,) (36,) (36, 4)
3 -> f: (72,) (72,) (72, 4)
4 -> m1,m2: 36 36 1.0 1.0
5 -> f=f+l: (72,) : [(345002, 4), (105001, 4), (210001, 4), (225002, 4), (130001, 4), (65001, 4), (195001, 4), (65001, 4), (130001, 4), (195001, 4), (65001, 4), (130001, 4), (225002, 4), (65001, 4), (130001, 4), (195001, 4), (65001, 4), (130001, 4), (75001, 4), (130001, 4), (195001, 4), (195001, 4), (130001, 4), (65001, 4), (65001, 4), (130001, 4), (195001, 4), (195001, 4), (130001, 4), (65001, 4), (65001, 4), (130001, 4), (195001, 4), (130001, 4), (195001, 4), (65001, 4), (345002, 4), (105001, 4), (210001, 4), (225002, 4), (130001, 4), (65001, 4), (195001, 4), (65001, 4), (130001, 4), (195001, 4), (65001, 4), (130001, 4), (225002, 4), (65001, 4), (130001, 4), (195001, 4), (65001, 4), (130001, 4), (75001, 4), (130001, 4), (195001, 4), (195001, 4), (130001, 4), (65001, 4), (6

0 2 0 4
Found precomputed model of 0, tested on 1. Testing on 4...
0 2 0 5
Found precomputed model of 0, tested on 1. Testing on 5...
0 2 1 0
Fitting on 1, testing on 0...
0 2 1 1
0 0 1 2
Found precomputed model of 1, tested on 0. Testing on 2...
0 0 1 3
Found precomputed model of 1, tested on 0. Testing on 3...
0 0 1 4
Found precomputed model of 1, tested on 0. Testing on 4...
0 0 1 5
Found precomputed model of 1, tested on 0. Testing on 5...
0 0 2 0
Fitting on 2, testing on 0...
0 0 2 1
Found precomputed model of 2, tested on 0. Testing on 1...
0 0 2 2
0 1 0 1
Fitting on 0, testing on 1...
0 0 2 3
Found precomputed model of 2, tested on 0. Testing on 3...
0 0 2 4
Found precomputed model of 2, tested on 0. Testing on 4...
0 0 2 5
Found precomputed model of 2, tested on 0. Testing on 5...
0 0 3 0
Fitting on 3, testing on 0...
0 0 3 1
Found precomputed model of 3, tested on 0. Testing on 1...
0 0 3 2
Found precomputed model of 3, tested on 0. Testing on 2...
0 0 3 3
0 1 0 2
Found precom

In [ ]:
############ RESULT REPORTING ##############
# generate files with stats
bargraph_perf_gen1(6)
bargraph_perf_gen2(6)
bargraph_perf_gen3(6)
bargraph_perf_gen4(6)
bargraph_perf_gen5(6)
# use the bargraph tool to plot graphs from generated files
# -left column cross-accuracy (trained on one, tested on all the others), 
# -right column self-accuracy (trained and tested on the same)
# -each row i represents training only with i surfaces.
# -each stack represents a training group, each bar represents a subfeatureset(AFFT,FREQ,TIME,BOTH)
# -blue,green,yellow,red : TP,TN,FN,FP
plt.figure(figsize=(20,40))
for i in range(5):
    make_bargraphs_from_perf(i)

In [ ]:
############ ONLINE TESTING PROCEDURE ##############
# same necessary steps as in training for data preparation
f,l,fd,member,m1,m2 = data_prep(c.validfile)
prefeat = compute_prefeat(f)
features, labels = feature_extraction(prefeat, member, c.validfeatfile, 'validfeat_')
new_labels = label_cleaning(prefeat,labels,member)
X,Y,Yn,Xsp,Ysp = computeXY(features,labels,new_labels,m1,m2,c.validXYfile,c.validXYsplitfile)
surf, surfla = computeXY_persurf(Xsp,Ysp,c.validsurffile)

In [ ]:
############ VISUALIZING ONLINE TESTING PROCEDURE ##############
window=c.window
subfeats = ['AFFT','FREQ','TIME','BOTH']
feats = ['fnorm','ftfn','fnormftfn']
matplotlib.rcParams['text.usetex'] = True

fileid = filename1(0,3,0,5)
fileidb = filename1(0,0,0,5)
fileid5 = filename5(0,3,0,1,2,3,4,5)
fileid5b = filename5(0,0,0,1,2,3,4,5)
model = np.load(fileid)['model'][0]
modelb = np.load(fileidb)['model'][0]
model5 = np.load(fileid5)['model'][0]
model5b = np.load(fileid5b)['model'][0]
Yout = model.predict(X[0])
Youtb = modelb.predict(Xsp[0][:,-window-2:-window/2-1])
Yout5 = model5.predict(Xsp[0])
Yout5b = model5b.predict(Xsp[0][:,-window-2:-window/2-1])
print Yout.shape, Yout5.shape, Yout5b.shape
plt.rc('text', usetex=True)
plt.rc('axes', linewidth=2)
plt.rc('font', weight='bold')
plt.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']
offset = 2000-window
endset = 2650
skipf = 20
skipy = 15
ax = plt.figure(figsize=(20,10))
tf = np.linalg.norm(f[0][offset+window::skipf,:3][:endset],axis=1)
p1, = plt.plot(tf/max(tf),linewidth=5)
ty = Yout[offset/skipf:][:endset]+0.02
print tf.shape, ty.shape
p = plt.scatter(range(len(tf))[::skipy],ty[::skipy],color='red',s=30)
plt.hold
plt.text(100, 0.15, r'\textbf{Stable}', ha="center", va="center", rotation=0,
            size=25)
plt.text(1000, 0.85, r'\textbf{Slip}', ha="center", va="center", rotation=0,
            size=25)
plt.annotate('', fontsize=10, xy=(100, 0.05), xytext=(100, 0.12),
            arrowprops=dict(facecolor='black', shrink=0.05))
plt.annotate('', xy=(1000, 0.98), xytext=(1000, 0.9),
            arrowprops=dict(facecolor='black', shrink=0.05))
plt.text(400, 0.55, r'\textbf{P1}', ha="center", va="center", rotation=0,
            size=25)
plt.axvline(x=810,linestyle='dashed',color='black',linewidth=5)
plt.text(1000, 0.55, r'\textbf{P2}', ha="center", va="center", rotation=0,
            size=25)
plt.axvline(x=1200,linestyle='dashed',color='black',linewidth=5)
plt.text(1250, 0.55, r'\textbf{P3}', ha="center", va="center", rotation=0,
            size=25)
plt.axvline(x=1335,linestyle='dashed',color='black',linewidth=5)
plt.text(1385, 0.25, r'\textbf{P4}', ha="center", va="center", rotation=0,
            size=25)
plt.axvline(x=1445,linestyle='dashed',color='black',linewidth=5)
plt.text(1650, 0.55, r'\textbf{P1}', ha="center", va="center", rotation=0,
            size=25)
plt.axvline(x=1830,linestyle='dashed',color='black',linewidth=5)
plt.text(2000, 0.55, r'\textbf{P2}', ha="center", va="center", rotation=0,
            size=25)
plt.axvline(x=2200,linestyle='dashed',color='black',linewidth=5)
plt.text(2250, 0.55, r'\textbf{P3}', ha="center", va="center", rotation=0,
            size=25)
plt.axvline(x=2330,linestyle='dashed',color='black',linewidth=5)
plt.text(2385, 0.25, r'\textbf{P4}', ha="center", va="center", rotation=0,
            size=25)
plt.axvline(x=2440,linestyle='dashed',color='black',linewidth=5)
plt.text(2540, 0.55, r'\textbf{P1}', ha="center", va="center", rotation=0,
            size=25)
plt.xlabel(r't ($1e^{-2} sec$)',fontsize=35)
# plt.yticks([])
plt.legend([p1,p],[r'$|\textbf{f}|$',r'\textbf{out1}'],loc=2, prop={'size': 35})
plt.tick_params(labelsize=20)
plt.tight_layout()
savefig(c.datapath+'validation.pdf', bbox_inches='tight')


In [ ]:
####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
prediction('ati_new_fd1.0N_kp3.5_152Hz_validation.mat')
####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
prediction('ati_new_fd1.0N_kp3.5_152Hz_validation_rot.mat')

In [ ]:
####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
prediction('ati_new_fd1.5N_kp3_152Hz_validation.mat')
####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
prediction('ati_new_fd1.5N_kp3_152Hz_validation_rot.mat')

In [ ]:
####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
prediction('ati_new_fd1.0N_kp3.5_326Hz_validation.mat')
####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
prediction('ati_new_fd1.0N_kp3.5_326Hz_validation_rot.mat')

In [ ]:
####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
prediction('ati_new_fd1.5N_kp3_326Hz_validation.mat')
####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
prediction('ati_new_fd1.5N_kp3_326Hz_validation_rot.mat')

In [ ]:
####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
prediction('ati_new_fd1.0N_kp3.5_836Hz_validation.mat')
####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
prediction('ati_new_fd1.0N_kp3.5_836Hz_validation_rot.mat')

In [ ]:
####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
prediction('ati_new_fd1.5N_kp3.5_836Hz_validation.mat')
####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
prediction('ati_new_fd1.5N_kp3.5_836Hz_validation_rot.mat')

In [ ]:
####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
prediction('ati_new_fd1N_kp3_nofilt_validation.mat')
####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
prediction('ati_new_fd1N_kp3_nofilt_validation_rot.mat')

In [ ]:
####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
prediction('ati_new_fd1.5N_kp3_nofilt_validation.mat')
####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
prediction('ati_new_fd1.5N_kp3_nofilt_validation_rot.mat')

## Scaled Version of one of the above on the following ranges (0.1, 0.5, 1.0, 2.0, 5.0, 20.0)

In [ ]:
for sc in [0.1, 0.5, 1.0, 2.0, 5.0, 20.0]:
    print "-------- SCALING = ",sc,"--------"
    ####### NEWER TESTING DATA FROM ATI F/T SENSOR TRANSLATIONAL CASE
    prediction('ati_new_fd1.5N_kp3_152Hz_validation.mat',scale=sc)
    ####### NEWER TESTING DATA FROM ATI F/T SENSOR ROTATIONAL CASE
    prediction('ati_new_fd1.5N_kp3_152Hz_validation_rot.mat',scale=sc)

## Check the training forces and compare them with testing ones (2.86 1.35 2.12 1.68)

In [ ]:
printit = False
f,_,_,_,_,_ = data_prep(c.datafile,printit=printit)                        # read training input force
pf = compute_prefeat(f,printit=printit)                                  # compute corresponding prefeatures
fv,_,_,_,_,_ = data_prep(c.validfile,printit=printit)                      # read validation input force
pfv = compute_prefeat(fv,printit=printit)                                # compute corresponding prefeatures

atifiles = ['ati_new_fd1.0N_kp3.5_152Hz_validation.mat',
            'ati_new_fd1.0N_kp3.5_152Hz_validation_rot.mat',
            'ati_new_fd1.5N_kp3_152Hz_validation.mat',
            'ati_new_fd1.5N_kp3_152Hz_validation_rot.mat',
            'ati_new_fd1.0N_kp3.5_326Hz_validation.mat',
            'ati_new_fd1.0N_kp3.5_326Hz_validation_rot.mat',
            'ati_new_fd1.5N_kp3_326Hz_validation.mat',
            'ati_new_fd1.5N_kp3_326Hz_validation_rot.mat',
            'ati_new_fd1.0N_kp3.5_836Hz_validation.mat',
            'ati_new_fd1.0N_kp3.5_836Hz_validation_rot.mat',
            'ati_new_fd1.5N_kp3.5_836Hz_validation.mat',
            'ati_new_fd1.5N_kp3.5_836Hz_validation_rot.mat',
            'ati_new_fd1N_kp3_nofilt_validation.mat',
            'ati_new_fd1N_kp3_nofilt_validation_rot.mat',
            'ati_new_fd1.5N_kp3_nofilt_validation.mat',
            'ati_new_fd1.5N_kp3_nofilt_validation_rot.mat']
atiftr = []
atifrt = []
for filen in atifiles:
    tf,_,_,_,_,_ = data_prep(c.datapath+filen,k=1,printit=printit)
    ptf = compute_prefeat(tf,printit=printit)
    if filen[-7:-4] == 'rot':
        atifrt.append(ptf)
    else:
        atiftr.append(ptf)
atiftr = np.array(atiftr).flatten()
atifrt = np.array(atifrt).flatten()

plist = [pf, pfv, atiftr, atifrt]
pname = ['train', 'valid','atitran','atirot']
print pf.shape, pfv.shape, atiftr.shape, atifrt.shape
mf, mfst, mfsl = np.zeros((4,4)), np.zeros((4,4)), np.zeros((4,4))
print 'datasetname: [all/stable/slip:[0:mean, 1:max, 2:min, 3:std]]'
for ind in range(len(plist)):
    pt = plist[ind]
    # 0:mean, 1:max, 2:min, 3:std
    for p in range(len(pt)):
        mf[ind,0] += np.mean(pt[p][:,0])
        mf[ind,1] += np.max(pt[p][:,0])
        mf[ind,2] += np.min(pt[p][:,0])
        mf[ind,3] += np.std(pt[p][:,0])
        stind = pt[p][:,1]==0
        slind = pt[p][:,1]==1
        mfst[ind,0] += np.mean(pt[p][stind,0])
        mfst[ind,1] += np.max(pt[p][stind,0])
        mfst[ind,2] += np.min(pt[p][stind,0])
        mfst[ind,3] += np.std(pt[p][stind,0])
        mfsl[ind,0] += np.mean(pt[p][slind,0])
        mfsl[ind,1] += np.max(pt[p][slind,0])
        mfsl[ind,2] += np.min(pt[p][slind,0])
        mfsl[ind,3] += np.std(pt[p][slind,0])
    mf[ind,0] /= len(plist[ind])
    mf[ind,1] /= len(plist[ind])
    mf[ind,2] /= len(plist[ind])
    mf[ind,3] /= len(plist[ind])
    mfst[ind,0] /= len(plist[ind])
    mfst[ind,1] /= len(plist[ind])
    mfst[ind,2] /= len(plist[ind])
    mfst[ind,3] /= len(plist[ind])
    mfsl[ind,0] /= len(plist[ind])
    mfsl[ind,1] /= len(plist[ind])
    mfsl[ind,2] /= len(plist[ind])
    mfsl[ind,3] /= len(plist[ind])
    print pname[ind], mf[ind], mfst[ind], mfsl[ind]